In [1]:
import torch
if torch.cuda.is_available():
  device = torch.device("cuda")
  print('There are %d GPU(s) available.' % torch.cuda.device_count())
  print('We will use the GPU:', torch.cuda.get_device_name(0))
else:
  print('No GPU availvable, using CPU instead.')
  device = torch.device("cpu")

There are 2 GPU(s) available.
We will use the GPU: Tesla T4


In [2]:
!pwd

/kaggle/working


In [3]:
!ls /kaggle/input

paraphrases  ruwikiruscorpora-upos-cbow


In [4]:
!tree /kaggle/input

/kaggle/input
├── paraphrases
│   ├── checkpoint_epoch3_step120.pt
│   ├── data.json
│   ├── pairs.tsv
│   └── pairs_reduced.tsv
└── ruwikiruscorpora-upos-cbow
    └── ruwikiruscorpora_upos_cbow_300_10_2021.bin

2 directories, 5 files


In [5]:
%%capture
! pip install sentencepiece
! pip install transformers
! pip install "treesimi>=0.1.5" datasketch
! pip install -U pip setuptools wheel
! pip install -U spacy
! python -m spacy download ru_core_news_sm
! pip install accelerate -U
! pip install optuna

In [6]:
import json

with open('/kaggle/input/paraphrases/data.json', 'r') as f:
    data = json.load(f)

In [7]:
# Use train_test_split to split our data into train and validation sets for
# training
from sklearn.model_selection import train_test_split

# Use 90% for training and 10% for validation.
train_dataset, validation_dataset = train_test_split(data, random_state=2018, test_size=0.1)

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [8]:
len(train_dataset)

3615

In [9]:
# Print the first few elements of train_dataset and validation_dataset
print("Train Dataset Sample:")
print(train_dataset[:2])  # Print the first 2 elements

print("\nValidation Dataset Sample:")
print(validation_dataset[:2])  # Print the first 2 elements

Train Dataset Sample:
[{'input': 'Вы что то плохо выглядите что у вас что-нибудь болит', 'target': 'У вас голос странный что у вас что-нибудь болит', 'input_tree': [570272283, 353300086, 122879569, 999870266, 157978476, 1902875321, 729333537, 444010880, 1087209578, 116765198, 248846295, 219859497, 394809467, 2034940150, 84077671, 160454645, 380451629, 943823497, 1099385344, 279935846, 316940651, 1257837434, 63178866, 159543812, 485744649, 60959675, 91280724, 383174313, 85663715, 813665748, 423576522, 306727142, 293869808, 30364140, 112312715, 551342760, 461167410, 396389078, 150082588, 658737846, 14637594, 191833, 986910865, 425065566, 20059047, 120551759, 519959666, 242155070, 820925023, 83736172, 184450484, 1709726678, 1224230697, 312347740, 3210310, 500055946, 136922280, 363677623, 90468210, 843661850, 44997228, 609310021, 672674213, 35531536, 735606613, 188509787, 256816523, 146092238, 1045013226, 1105791047, 488705254, 245785279, 923583071, 176964394, 80252480, 143403444, 40216943

In [10]:
import os
import json
import subprocess
import datasketch
import numpy as np
from transformers import T5ForConditionalGeneration, T5Tokenizer, TrainingArguments, EarlyStoppingCallback
from transformers import MT5ForConditionalGeneration, AutoTokenizer
from sklearn.metrics.pairwise import cosine_similarity
from gensim.models import KeyedVectors
from torch.utils.data import DataLoader, Dataset
import random
from sklearn.model_selection import train_test_split
from scipy.spatial.distance import jaccard
from tqdm import tqdm
import torch
import torch.nn as nn
import torch.nn.functional as F
from nltk.translate.bleu_score import corpus_bleu, SmoothingFunction

In [11]:
# Define a custom dataset
class CustomDataset(Dataset):
    def __init__(self, examples, tokenizer, max_length):
        self.examples = examples
        self.tokenizer = tokenizer
        self.max_length = max_length

    def __len__(self):
        return len(self.examples)

    def __getitem__(self, idx):
        example = self.examples[idx]
        input_tree = example["input_tree"]
        target_tree = example["target_tree"]
        input_text = "paraphrase: " + example["input"]
        target_text = example["target"]

        # Calculate syntactic similarity
        syntactic_similarity = calculate_syntactic_similarity(input_tree, target_tree)

        # Calculate semantic similarity
        semantic_similarity = calculate_semantic_similarity(input_text, target_text)

        # Tokenize input and target texts
        inputs = self.tokenizer.batch_encode_plus([input_text], return_tensors="pt", max_length=self.max_length, padding="max_length", truncation=True)
        targets = self.tokenizer.batch_encode_plus([target_text], return_tensors="pt", max_length=self.max_length, padding="max_length", truncation=True)

        return {
            "input_texts": input_text,
            "input_ids": inputs["input_ids"].squeeze(),
            "attention_mask": inputs["attention_mask"].squeeze(),
            "decoder_input_ids": targets["input_ids"].squeeze(),
            "labels": targets["input_ids"].squeeze(),
            "decoder_attention_mask": targets["attention_mask"].squeeze(),
            "target_syntactic_similarities": syntactic_similarity,
            "target_semantic_similarities": semantic_similarity,
        }

# Split our data into train and validation sets
train_data, validation_data = train_test_split(data, random_state=2018, test_size=0.1)

# Define the function to calculate syntactic similarity
def calculate_syntactic_similarity(tree1, tree2):
    tree_set1 = set(tree1)
    tree_set2 = set(tree2)
    intersection = len(tree_set1.intersection(tree_set2))
    union = len(tree_set1.union(tree_set2))
    jaccard_similarity = intersection / union
    return jaccard_similarity

# Load the pre-trained Word2Vec model
word2vec_model_path = '/kaggle/input/ruwikiruscorpora-upos-cbow/ruwikiruscorpora_upos_cbow_300_10_2021.bin'
word2vec_model = KeyedVectors.load_word2vec_format(word2vec_model_path, binary=True)

# Define the function to calculate the vector representation of a sentence using Word2Vec
def calculate_word2vec_embedding(text):
    words = text.split()
    vectors = [word2vec_model.get_vector(word) for word in words if word in word2vec_model.key_to_index]
    if vectors:
        return np.mean(vectors, axis=0)
    else:
        return np.zeros(word2vec_model.vector_size)

# Define the function to calculate BERT embeddings for semantic similarity
def calculate_semantic_embeddings(texts):
    return np.array([calculate_word2vec_embedding(text) for text in texts])

# Define the function to calculate semantic similarity
def calculate_semantic_similarity(input, paraphrase):
    input_embedding = calculate_semantic_embeddings([input])
    paraphrase_embedding = calculate_semantic_embeddings([paraphrase])
    semantic_similarity = cosine_similarity(input_embedding, paraphrase_embedding).squeeze()
    normalized_semantic_similarity = (semantic_similarity + 1) / 2
    return normalized_semantic_similarity

# Load the pre-trained model and tokenizer
MODEL_NAME = 'cointegrated/rut5-base-paraphraser'
model = T5ForConditionalGeneration.from_pretrained(MODEL_NAME)
model.to(device)
tokenizer = T5Tokenizer.from_pretrained(MODEL_NAME)

# Create custom datasets
max_length = 128  # Adjust as needed
train_dataset = CustomDataset(train_data, tokenizer, max_length)
validation_dataset = CustomDataset(validation_data, tokenizer, max_length)

# Define data loaders
train_batch_size = 16  # Adjust as needed (8 initially)
validation_batch_size = 16  # Adjust as needed (16 initially)

train_dataloader = DataLoader(
    train_dataset,
    batch_size=train_batch_size,
    shuffle=True,  # Shuffle the data at the beginning of each epoch
    num_workers=4,  # Adjust the number of workers based on your system's capabilities
)

validation_dataloader = DataLoader(
    validation_dataset,
    batch_size=validation_batch_size,
    shuffle=False,  # No need to shuffle validation data
    num_workers=4,  # Adjust the number of workers based on your system's capabilities
)


def custom_loss(syntactic_similarities, target_syntactic_similarities,
                semantic_similarities, target_semantic_similarities,
                model_outputs, alpha=0.8, beta=0.5, gamma=0.3, device=device):
    # Calculate the syntactic similarity loss
    syntactic_loss = nn.MSELoss()(syntactic_similarities, target_syntactic_similarities.detach()).to(device)

    # Calculate the semantic similarity loss
    semantic_loss = nn.MSELoss()(semantic_similarities, target_semantic_similarities.detach()).to(device)

    # Extract the model's loss from the outputs
    model_loss = model_outputs.loss

    # Combine the losses using weights
    total_loss = alpha * syntactic_loss + beta * semantic_loss + gamma * model_loss

    return total_loss
    
def generate_paraphrases(input_ids, model, tokenizer, max_length=128, device="cuda", training=True):
    # Generate paraphrase
    if training:
        output_sequences = model.generate(input_ids=input_ids, max_length=max_length, num_return_sequences=1).to(device)
    else:
        with torch.no_grad():  # If validation or testing, gradients are not calculated
            output_sequences = model.generate(input_ids=input_ids, max_length=max_length, num_return_sequences=1).to(device)
    
    # Decode the generated paraphrase
    paraphrases = tokenizer.batch_decode(output_sequences, skip_special_tokens=True)

    return paraphrases


# Define fine-tuning arguments
training_args = TrainingArguments(
    output_dir="./output",
    num_train_epochs=5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    learning_rate=3e-5,
    weight_decay=1e-2, 
    evaluation_strategy="steps",
    save_steps=1000,
    eval_steps=1000,
    save_total_limit=3,
    remove_unused_columns=False,
)

# Initialize the optimizer and the scheduler
optimizer = torch.optim.AdamW(model.parameters(), lr=training_args.learning_rate, weight_decay=training_args.weight_decay)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=1, gamma=0.95)

# Define the checkpoint directory
checkpoint_dir = training_args.output_dir

# Create the directory if it doesn't exist
if not os.path.exists(checkpoint_dir):
    os.makedirs(checkpoint_dir)

# Check if a checkpoint exists
checkpoint_file = "/kaggle/input/paraphrases/checkpoint_epoch3_step320.pt"  # Change this to the path of your checkpoint file
if os.path.exists(checkpoint_file):
    checkpoint = torch.load(checkpoint_file)
    model.load_state_dict(checkpoint['model_state_dict'])
    optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
    start_epoch = checkpoint['epoch']  # Use the epoch from the checkpoint
    total_loss = checkpoint['loss']
    num_batches = checkpoint['num_batches']  # Start from the next batch

    # Increment the epoch if all batches were processed
    if num_batches == len(train_dataloader):
        start_epoch += 1
        num_batches = 0
else:
    start_epoch = 0  # Start training from the beginning
    num_batches = 0

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. If you see this, DO NOT PANIC! This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thouroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565
/opt/conda/lib/python3.10/site-packages/torch/utils/data/dataloader.py:561: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(


In [12]:
# Initialize an empty list to store the validation results
validation_results = []

# Start fine-tuning from the specified epoch or 0 if no checkpoint was loaded
for epoch in range(start_epoch, training_args.num_train_epochs):
    print()
    print('Training...')

    model.train()
    if num_batches == 0:
        total_loss = 0.0

    # Create a tqdm progress bar for the training loop
    pbar = tqdm(train_dataloader, desc=f"Epoch {epoch + 1}/{training_args.num_train_epochs}")

    save_interval = 40
    checkpoint_dir_name = 'output'

    # Change the range to start from the next batch
    for batch_idx, batch in enumerate(pbar):
        if batch_idx < num_batches:
            continue  # Skip batches that were processed in the previous epoch
        
        # Checkpoint model and optimizer every 'save_interval' steps
        if (num_batches % save_interval) == 0:
            print()
            print("Checkpoint...")
            
            # Save the current checkpoint
            checkpoint_path = f"{checkpoint_dir_name}/checkpoint_epoch{epoch + 1}_step{num_batches}.pt"
            torch.save({
                'epoch': epoch,
                'model_state_dict': model.state_dict(),
                'optimizer_state_dict': optimizer.state_dict(),
                'loss': total_loss,
                'num_batches': batch_idx,  # Save the current batch index
            }, checkpoint_path)
            
            # Delete the previous checkpoint file if it exists
            if os.path.exists(checkpoint_file) and not checkpoint_file.startswith('/kaggle/input'):
                os.remove(checkpoint_file)
            
            # Update the checkpoint_file variable to the new checkpoint
            checkpoint_file = checkpoint_path
        
        input_texts = batch["input_texts"]
        input_ids = batch["input_ids"].to(device)
        attention_mask = batch["attention_mask"].to(device)
        decoder_input_ids = batch["decoder_input_ids"].to(device)
        labels = batch["labels"].to(device)
        decoder_attention_mask = batch["decoder_attention_mask"].to(device)
        syntactic_similarities_original = batch["target_syntactic_similarities"].to(device, dtype=torch.float64)
        semantic_similarities_original = batch["target_semantic_similarities"].to(device, dtype=torch.float64)
        
        # Do not force the model to predict pad tokens
        labels[labels == 0] = -100
        
        # Forward pass
        outputs = model(input_ids=input_ids, attention_mask=attention_mask, decoder_input_ids=decoder_input_ids,
                        decoder_attention_mask=decoder_attention_mask, labels=labels)

        # Generate paraphrases
        generated_texts = generate_paraphrases(input_ids, model, tokenizer, max_length=128, device=device)

        # Calculate syntactic similarities
        syntactic_similarities_generated = [calculate_syntactic_similarity(generated_text, input_text) for generated_text, input_text in zip(generated_texts, input_texts)]
        # Calculate semantic similarities
        semantic_similarities_generated = [calculate_semantic_similarity(generated_text, input_text) for generated_text, input_text in zip(generated_texts, input_texts)]

        # Convert similarities to tensors
        syntactic_similarities_generated = torch.tensor(syntactic_similarities_generated, requires_grad=True).to(device, dtype=torch.float64)
        semantic_similarities_generated = torch.tensor(semantic_similarities_generated, requires_grad=True).to(device, dtype=torch.float64)
        
        # Calculate loss manually
        loss = custom_loss(syntactic_similarities_generated, syntactic_similarities_original,
                           semantic_similarities_generated, semantic_similarities_original,
                           outputs)

        total_loss += loss.item()
        
        # Backpropagation
        loss.backward()
        
        optimizer.step()
        scheduler.step()
        optimizer.zero_grad()  # Clear gradients for the next batch
        num_batches += 1

        # Update tqdm progress bar description with batch loss
        pbar.set_postfix({"Batch Loss": loss.item(), "Avg Loss": total_loss / num_batches})

    avg_loss = total_loss / num_batches
    print(f"Epoch {epoch + 1}/{training_args.num_train_epochs}, Avg Loss: {avg_loss}")

    # Evaluate on validation set

    print()
    print("Running Validation...")

    model.eval()
    num_batches = 0
    bleu_scores = []
    results = []

    for batch in tqdm(validation_dataloader, desc="Validation"):
        input_texts = batch["input_texts"]
        input_ids = batch["input_ids"].to(device)
        attention_mask = batch["attention_mask"].to(device)

        # Telling the model not to compute or store gradients, saving memory and
        # speeding up validation
        generated_paraphrases = generate_paraphrases(input_ids, model, tokenizer, max_length, device=device, training=False)
        start = num_batches * validation_batch_size
        end = (num_batches + 1) * validation_batch_size
        reference_paraphrases = [example['target'] for example in validation_data[start:end]]

        print()
        print('inputs', input_texts)
        print('generated_paraphrases', generated_paraphrases)
        print('targets', reference_paraphrases)
        
        # Store the reference paraphrases and generated paraphrases in the dictionary
        for reference_text, generated_text in zip(reference_paraphrases, generated_paraphrases):
            results.append({
                "reference_paraphrase": reference_text,
                "generated_paraphrase": generated_text
            })

        # Define a SmoothingFunction
        smoother = SmoothingFunction()

        # Calculate BLEU score
        batch_bleu_score = corpus_bleu(reference_paraphrases, generated_paraphrases, smoothing_function=smoother.method1)
        bleu_scores.append(batch_bleu_score)

        num_batches += 1

    # Calculate average validation loss
    avg_bleu_score = sum(bleu_scores) / len(bleu_scores)
    print(f"Validation Avg BLEU Score for Epoch {epoch + 1}: {avg_bleu_score}")
    num_batches = 0

    # Create a dictionary to store the validation results for this epoch
    epoch_result = {
        "epoch": epoch + 1,
        "avg_bleu_score": avg_bleu_score,
        "results": results
    }

    # Append the epoch results to the list
    validation_results.append(epoch_result)

    # Save the validation results as a JSON file
    with open('/kaggle/working/output/validation_results.json', 'w', encoding='utf-8') as json_file:
        json.dump(validation_results, json_file, ensure_ascii=False, indent=4)

# Save the final model
final_model_path = f'{checkpoint_dir}/fine_tuned_model'
model.save_pretrained(final_model_path)

print()
print("Training complete!")


Training...


Epoch 1/5:   0%|          | 0/226 [00:00<?, ?it/s]


Checkpoint...


Epoch 1/5:  18%|█▊        | 40/226 [01:17<06:29,  2.09s/it, Batch Loss=2.35, Avg Loss=2.52]


Checkpoint...


...


inputs ['paraphrase: В проекте трехлетнего бюджета обнаружилась "дыра" в 250 млрд руб.', 'paraphrase: Владимир Путин поздравил актера Владимира Этуша с 93-летием', 'paraphrase: Ну ка признавайся как ты ко мне относишься', 'paraphrase: обожаю тебя уже а ты обо мне хоть что нибудь знаешь', 'paraphrase: Полиция задержала водителя микроавтобуса, сгоревшего под Самарой.', 'paraphrase: Полиция провела более 20 обысков в московских банках.', 'paraphrase: Взрыв в иракском кафе унес жизни 38 человек.', 'paraphrase: Есть ли у тебя дети?', 'paraphrase: Тебе нравится человечество?', 'paraphrase: Что ты делала в моё отсутствие?', 'paraphrase: После опрокидывания автобуса в Тюменской области ранены 14 человек', 'paraphrase: Спроси у меня что нибудь!', 'paraphrase: Омские приставы избили полицейского.', 'paraphrase: Тебе нравится читать или ты ненавидишь это делать?', 'paraphrase: США исключили Кубу из списка стран-спонсоров терроризма', 'paraphrase: Депутатам разрешили обходиться без присяги на вер

Validation: 100%|██████████| 26/26 [00:19<00:00,  1.65it/s]


inputs ['paraphrase: у меня есть любимая собака', 'paraphrase: какая у тебя любимая марка машины?']
generated_paraphrases ['Тема: у меня есть любимая собака', 'Вопрос: какая у вас любимая марка машины?']
targets ['у меня есть гавкающее животное', 'Слушай какая у тебя любимая марка машины?']


Validation: 100%|██████████| 26/26 [00:20<00:00,  1.30it/s]


Validation Avg BLEU Score for Epoch 5: 0.0010845472663644565

Training complete!


In [13]:
test_dataset = [{"input": "Каждый охотник желает знать, где сидит фазан.", "target": "Все охотники хотят знать, где фазан сидит."}]

# Load the fine-tuned model
fine_tuned_model = T5ForConditionalGeneration.from_pretrained('./output/fine_tuned_model')

# Testing
for example in test_dataset:
    input_text = example['input']
    target_text = example['target']

    # Tokenize input text
    input_ids = tokenizer.encode(input_text, return_tensors='pt')

    # Generate paraphrased output
    generated_ids = fine_tuned_model.generate(input_ids)

    # Decode generated output
    generated_text = tokenizer.decode(generated_ids[0], skip_special_tokens=True)

    # Print input, target, and generated output
    print(f"Input: {input_text}")
    print(f"Target: {target_text}")
    print(f"Generated: {generated_text}")
    print("=" * 30)

/opt/conda/lib/python3.10/site-packages/transformers/generation/utils.py:1260: UserWarning: Using the model-agnostic default `max_length` (=20) to control thegeneration length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


Input: Каждый охотник желает знать, где сидит фазан.
Target: Все охотники хотят знать, где фазан сидит.
Generated: Каждый охотник хочет знать, где сидит фазан.
